In [2]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [5]:
!pip install tflearn 

     |████████████████████████████████| 112kB 2.8MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127300 sha256=b2f03fd2f5b1f720e896f3abed80f5ce776c93d7c28c702232c6bcbbb4ac20d7
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [6]:
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up"],\r\n         "responses": ["Hello!", "Good to see you again!", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day"],\r\n         "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "age",\r\n         "patterns": ["how old", "how old is tim", "what is your age", "how old are you", "age?"],\r\n         "responses": ["I am 18 years old!", "18 years young!"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "name",\r\n         "patterns": ["what is your name", "what should I call you", "whats your name?"],\r\n         "responses": ["You can call me Tim.", "I\'m Tim!", "I\'m Tim aka Tech With

In [8]:
with open("intents.json") as json_data:
  intents = json.load(json_data)

In [9]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'context_set': '',
   'patterns': ['cya',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day'],
   'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
   'tag': 'goodbye'},
  {'context_set': '',
   'patterns': ['how old',
    'how old is tim',
    'what is your age',
    'how old are you',
    'age?'],
   'responses': ['I am 18 years old!', '18 years young!'],
   'tag': 'age'},
  {'context_set': '',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Tim.',
    "I'm Tim!",
    "I'm Tim aka Tech With Tim."],
   'tag': 'name'},
  {'context_set': '',
   'patterns': ['Id like to buy something',
    'whats on the men

In [11]:
words = []
classes = []
documents = []
ignore = ['?']

for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [23]:
documents

[(['Hi'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Whats', 'up'], 'greeting'),
 (['cya'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['I', 'am', 'Leaving'], 'goodbye'),
 (['Have', 'a', 'Good', 'day'], 'goodbye'),
 (['how', 'old'], 'age'),
 (['how', 'old', 'is', 'tim'], 'age'),
 (['what', 'is', 'your', 'age'], 'age'),
 (['how', 'old', 'are', 'you'], 'age'),
 (['age', '?'], 'age'),
 (['what', 'is', 'your', 'name'], 'name'),
 (['what', 'should', 'I', 'call', 'you'], 'name'),
 (['whats', 'your', 'name', '?'], 'name'),
 (['Id', 'like', 'to', 'buy', 'something'], 'shop'),
 (['whats', 'on', 'the', 'menu'], 'shop'),
 (['what', 'do', 'you', 'reccommend', '?'], 'shop'),
 (['could', 'i', 'get', 'something', 'to', 'eat'], 'shop'),
 (['when', 'are', 'you', 'guys', 'open'], 'hours'),
 (['what', 'are', 'your', 'hours'], 'hours'),
 (['hours', 'of',

In [26]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [27]:
print(len(documents), "documents")
print(len(classes), "documents", classes)
print(len(words), "unique steemed words", words)

26 documents
6 documents ['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']
46 unique steemed words ['a', 'ag', 'am', 'anyon', 'ar', 'buy', 'cal', 'could', 'cya', 'day', 'do', 'eat', 'get', 'good', 'goodby', 'guy', 'hav', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'is', 'lat', 'leav', 'lik', 'menu', 'nam', 'of', 'old', 'on', 'op', 'reccommend', 'see', 'should', 'som', 'the', 'ther', 'tim', 'to', 'up', 'what', 'when', 'yo', 'you']


In [41]:
training =[]
output = []

output_empty = [0] * len(classes)
output_empty

[0, 0, 0, 0, 0, 0]

In [46]:
for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row])

In [48]:
random.shuffle(training)
training = np.array(training)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [58]:
train_x = list(training[:,0])
train_y = list(training[:,1])


In [62]:
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [63]:
model = tflearn.DNN(net, tensorboard_dir="tflearn_logs")

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 12999  | total loss: 0.00378 | time: 0.117s
| Adam | epoch: 1000 | loss: 0.00378 - acc: 1.0000 -- iter: 096/104
Training Step: 13000  | total loss: 0.00369 | time: 0.123s
| Adam | epoch: 1000 | loss: 0.00369 - acc: 1.0000 -- iter: 104/104
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [64]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [stemmer.stem(w.lower()) for w in sentence_words]
  return sentence_words

In [74]:
def bag_of_words(sentence, words, show_details=False):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for s in sentence_words:
    for i, w in enumerate(words):
      if w == s:
        bag[i] = 1
        if show_details:
            print("found in bag: %s" %w)
  return(np.array(bag))
       


In [78]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = classes[results_index]

        for tg in intents["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hi
Hi there, how can I help?
You: i'm good
Hi there, how can I help?
You: buy somthing
We sell chocolate chip cookies for $2!
You: open
We are open 7am-4pm Monday-Friday!
You: byee
Hi there, how can I help?
You: good byee
Good to see you again!
You: quit
